In [2]:
# импорт библиотек
import sys
import os
import cv2
import re
import string
import copy
import datetime
import pandas as pd
import numpy as np
import Levenshtein as lev

from PIL import Image as PILImage

from pytesseract import Output
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

from img2table.document import Image as DocumentImage
from img2table.ocr import TesseractOCR
from img2table.ocr import EasyOCR
import easyocr

from IPython.display import display_html



class Form405:
    '''
    Класс для распознавания отметки о взятии крови из "Учетной карточки донора" (Форма № 405-05/у).
    Атрибуты класса:    
        folder                         директория с файлами
        image_file                     название исходного файла изображения с расширением
        image_file_is_exist            наличие исходного файла рисунка
        image_file_preprocessed        наличие обработки исходного файла рисунка
        table_extracted                извлечение структуры таблицы
        image_path                     путь к исходному файлу изображения           
        person_id                      префикс названия файлов, соответствующих определенному пользователю
        csv_path                       путь к исходному csv-файлу
        csv_recognised_path            путь к сгенерированному csv-файлу
        image_original                 исходный изображенное в обработке
        image_processed                преобразованное изображение в обработке
        image_processed_path           путь сохранения преобразованного изображения
        document_image                 объекта рисунка для разметки таблицы
        table_coords                   упорядоченный словарь с координатами ячеек
        n_rows                         число строк в таблице
        n_cols                         число столбцов в таблице
        x1, y1, x2, y2                 координаты левого верхнего и правого нижнего углов таблицы
        avr_cell_width                 расчетная средняя ширина ячеек
        avr_cell_height                расчетная средняя высота ячеек
        flag_split_cell                флаг расщепления ячеек
        flag_size_incorrected          флаг некорректного размера
        num_field                      число основных полей в таблице
        df_original                    исходный датафрейм
        df_recognised                  сгенерированный датафрейм
        accuracy                       точность распознавания
    
    Последовательность операций для запуска распознавания:
       1. Создание экземпляра класса и запуск предобработки
            obj = Form405('405/', '245365 .jpg')
       2. Получение разметки и параметров для таблицы
            obj.get_table_coords()
       3. Распознавание и сохранение файла с результатами
            obj.get_dataframe()
    '''

    # инициализация класса и назначение пути к файлу изображения
    def __init__(self, folder, file):
        '''
        Предполагаем, что программа будет работать с конкретной, заранее назначенной папкой. 
        Названия файлов соответствуют ID пользователя и имеют целочисленный формат.
        Новые файлы создаются поддиректории "new_files\".
        '''
        self.folder = None
        self.image_file = None
        self.image_path = None
        self.image_original = None
        self.image_file_is_exist = False
        self.image_file_preprocessed = False
        self.table_extracted = False
        self.image_original, self.image_processed = None, None
        self.image_processed_path = None
        self.document_image = None
        self.table_coords = None
        self.n_rows, self.n_cols = None, None 
        self.x1, self.y1, self.x2, self.y2 = None, None, None, None
        self.avr_cell_width, self.avr_cell_height = None, None
        self.flag_split_cell, self.flag_size_incorrected = None, None
        self.num_field = None
        self.df_original, self.df_recognised = None, None
        self.accuracy = None
        
        # проверка наличия указанного файла
        image_path = folder + file
        if os.path.exists(image_path) == False:
            print(f'Warning:1. Line:{sys._getframe().f_lineno}. Файл {file} отсутствует в рабочей директории!')
            return None
        # проверка считывания файла
        try:
            image_original = PILImage.open(image_path)
            # инициализация путей
            self.folder = folder
            self.image_file = file
            self.image_path = image_path
            self.image_original = image_original
            self.image_file_is_exist = True
        except:
            print(f'Warning:2. Line:{sys._getframe().f_lineno}. Файл {file} не является файлом изображения!')
            return None
        # получения префикса названия файла, соответствующего id пользователя
        try:
            self.person_id = re.match(r'(\D*)(\d+)(\D*)(.*)(\.)(\D*)', self.image_file).group(2)
        except:
            # id пользователя не является названием файля
            self.person_id = re.match(r'(.*)(\.)(\D*)', self.image_file).group(1)
        self.csv_path = self.folder + self.person_id + '.csv'
        self.csv_recognised_path = self.folder + self.person_id + '_recognised.csv'
        # запуск предобработки
        self.make_preprocessing()
        
        
         
#     функция предобработки изображения
#     def make_preprocessing(self):
#         if self.image_path is None:
#             print(f'Warning:3. Line:{sys._getframe().f_lineno}. Отсутствует файл для обработки!')
#             return
#         # попытка обработки
#         image_original = cv2.imread(self.image_path)
#         if image_original is None:
#             print(f'Warning:4. Line:{sys._getframe().f_lineno}. Файл {self.image_file} не обработан!')
#             return
#         self.image_original = image_original
#         self.image_file_preprocessed = True
#         # обрезка верхней части изображения
#         height = int(self.image_original.shape[0])
#         half_height = int(self.image_original.shape[0] / 2.)
#         self.image_processed = self.image_original[half_height:height, :]
#         # конвертирование цвета изображения в оттенки серого
#         self.image_processed = cv2.cvtColor(self.image_processed, cv2.COLOR_BGR2GRAY)
#         # добавление размытия для удаления шума
#         self.image_processed = cv2.medianBlur(self.image_processed, 1)
#         # введение порогового значения для сегментации изображения
#         self.image_processed = cv2.adaptiveThreshold(
#             self.image_processed, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 11
#         )
#         # сохранение файла
#         self.image_processed_path = self.folder + self.person_id + '_processed.png'
#         cv2.imwrite(self.image_processed_path, self.image_processed)
        
    # функция предобработки изображения
    def make_preprocessing(self):
        # проверка пути
        if self.image_path is None:
            print(f'Warning:3. Line:{sys._getframe().f_lineno}. Отсутствует файл для обработки!')
            return
        # попытка считывания
        image_original = cv2.imread(self.image_path)
        if image_original is None:
            print(f'Warning:4. Line:{sys._getframe().f_lineno}. Файл {self.image_file} не обработан!')
            return
        self.image_original = image_original
        self.image_file_preprocessed = True
        
        # (1) обработка оригинального изображения
        # (1.1) изменение размера до приемлемого
        desired_width = 3000
        # соотношение сторон: ширина, делённая на ширину оригинала
        aspect_ratio = desired_width / image_original.shape[1]
        # желаемая высота: высота, умноженная на соотношение сторон
        desired_height = int(image_original.shape[0] * aspect_ratio)
        # масштабирование изображения
        img = cv2.resize(image_original, dsize=(desired_width, desired_height), interpolation=cv2.INTER_NEAREST)
        width_original = int(img.shape[1])
        height_original = int(img.shape[0])        
        # (1.2) обрезка верхней части изображения
        half_height_original = int(img.shape[0] / 2.)
        img = img[half_height_original:height_original, :]        
        # (1.3) подготовка изображения к распознаванию
        # конвертирование цвета изображения в оттенки серого
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)        
        # добавление размытия для удаления шума
        img = cv2.medianBlur(img, 1)        
        # введение порогового значения для сегментации изображения
        img = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 11)

        # (2) обработка изображения для поиска контуров таблицы на копиии изображения
        # (2.1) создание копии изображения и ее подготовка
        # добавление размытия для удаления шума
        img_table = cv2.GaussianBlur(img,(5,5),0)
        # конверсия изображения в инверсивный черно-белый цвет
        _,img_table = cv2.threshold(img_table,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        # расширение\объединение светлых контуров
        kernel5 = np.ones((5, 5), 'uint8')
        img_table = cv2.dilate(img_table, kernel5, iterations=1)
        # удаление шумов
        kernel3 = np.ones((3, 3), 'uint8')
        img_table = cv2.morphologyEx(img_table, cv2.MORPH_OPEN, kernel3)
        # закрытие контуров
        kernel2 = np.ones((2, 2), 'uint8')
        img_table = cv2.morphologyEx(img_table, cv2.MORPH_CLOSE, kernel2)        
        # (2.2) определение границ таблицы
        contours, hierarchy = cv2.findContours(img_table, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        Xc1, Yc1, Xc2, Yc2 = 0, 0, img.shape[1], img.shape[0]
        for cnt in contours:
            X, Y, W, H = cv2.boundingRect(cnt)
            # определение границ таблицы
            if W > 0.6*width_original and H/2 > 0.06*width_original and X > 0 and Y > 0:
                Xc1, Xc2, Yc1, Yc2 = X, (X+W), Y, (Y+H)
        # (2.3) обрезка изображения и его копии под размер таблицы
        img_cropped = img[Yc1:Yc2, Xc1:Xc2]
        img_table_cropped = img_table[Yc1:Yc2, Xc1:Xc2]
        # размеры обрезанного изображения
        table_width = img_table_cropped.shape[1]
        table_height = img_table_cropped.shape[0]
        table_diag = (table_width**2 + table_height**2)**0.5
        
        # (3) определение углов таблицы для исправления перспективы
        # (3.1) определение угловых ячеек таблицы
        cell_contours, cell_hierarchy = cv2.findContours(img_table_cropped, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        corners = {'LU':[0, 0, table_diag], 'RU':[0, 0, table_diag],'LB':[0, 0, table_diag],'RB':[0, 0, table_diag]}
        for cell_cnt in cell_contours:
            x, y, w, h = cv2.boundingRect(cell_cnt)
            # поиск ячеек таблицы заданного размера
            if w > 0.05*table_width and w < 0.3*table_width and h > 0.01*table_width:
                # расстояния от углов ячеек до углов изображения
                LU_dist = ((0. - x)**2 + (0. - y)**2)**.5
                RU_dist = ((table_width - (x + w))**2 + (0. - y)**2)**.5
                LB_dist = ((0. - x)**2 + (table_height - (y + h))**2)**.5
                RB_dist = ((table_width - (x + w))**2 + (table_height - (y + h))**2)**.5
                # поиск ближайших к углам ячеек
                if LU_dist < corners['LU'][2]:
                    corners['LU'][2] = LU_dist
                    corners['LU'][0] = x
                    corners['LU'][1] = y
                if RU_dist < corners['RU'][2]:
                    corners['RU'][2] = RU_dist
                    corners['RU'][0] = x + w
                    corners['RU'][1] = y
                if LB_dist < corners['LB'][2]:
                    corners['LB'][2] = LB_dist
                    corners['LB'][0] = x
                    corners['LB'][1] = y + h
                if RB_dist < corners['RB'][2]:
                    corners['RB'][2] = RB_dist
                    corners['RB'][0] = x + w
                    corners['RB'][1] = y + h
        # (3.2) исправление перспективы (перекоса таблицы)
        # исходные и конечные координаты преобразования
        points_in = np.float32([corners['LU'][:2], corners['RU'][:2], corners['LB'][:2], corners['RB'][:2]])
        # добавление отступа от края изображения
        border_shift = 0.002*table_width
        points_out = np.float32([[border_shift, border_shift],
                                 [table_width - border_shift, border_shift],
                                 [border_shift,table_height - border_shift],
                                 [table_width - border_shift, table_height - border_shift]]
                               )
        # матрица преобразования
        transform_matrix = cv2.getPerspectiveTransform(points_in, points_out)
        # исправление перспективы
        img_cropped_corrected = cv2.warpPerspective(img_cropped, transform_matrix, (table_width,table_height))        

        # (4) сохранение файла
        self.image_processed = img_cropped_corrected
        self.image_processed_path = self.folder + self.person_id + '_processed.png'
        cv2.imwrite(self.image_processed_path, self.image_processed)  
        

    

    # функция определения структуры таблицы
    def __get_table_pos(self, table_extracted):
        # проверка налчия и предобработки файла изображения
        if self.image_file_is_exist == False or self.image_file_preprocessed == False or self.table_extracted == False:
            return
        # число элементов в таблице
        self.n_rows = len(table_extracted)
        self.n_cols = len(table_extracted[0])
        # координаты левого верхнего и правого нижнего углов таблицы
        self.x1 = table_extracted[0][0].bbox.x1
        self.y1 = table_extracted[0][0].bbox.y1
        self.x2 = table_extracted[self.n_rows-1][-1].bbox.x2
        self.y2 = table_extracted[self.n_rows-1][-1].bbox.y2
        # средний размер ячейки
        self.avr_cell_width = int((self.x2 - self.x1) / self.n_cols)
        self.avr_cell_height = int((self.y2 - self.y1) / self.n_rows)
        # проверка на сплит-ячейки
        self.flag_split_cell = False
        self.flag_size_incorrected = False
        for i in range(self.n_rows):
            if len(table_extracted[i]) != self.n_cols:
                self.flag_split_cols = True
        for i in range(self.n_rows):
            # проверка совпадения числа ячеек в строке
            if len(table_extracted[i]) != self.n_cols:
                    self.flag_split_cell = True
            # проверка размера ячеек
            for j in range(self.n_cols):
                current_cell_width = int(table_extracted[i][j].bbox.x2 - table_extracted[i][j].bbox.x1)
                current_cell_height = int(table_extracted[i][j].bbox.y2 - table_extracted[i][j].bbox.y1)
                if (
                    (abs(self.avr_cell_width - current_cell_width) > 0.1 * self.avr_cell_width)
                    or (abs(self.avr_cell_height - current_cell_height) > 0.1 * self.avr_cell_height)
                ):
                    self.flag_size_incorrected = True
    
    
 
    # функция получения таблицы с координатами ячеек
    def get_table_coords(self):
        '''
        Метод extract_tables некорректно распознаёт текст, записанный в несколько строк в шапке таблицы. 
        После чего он добавляет повторяющиеся соседние ячейки в столбцы. Такие дубли необходимо удалить.
        '''
        # проверка наличия и предобработки файла изображения
        if self.image_file_is_exist == False or self.image_file_preprocessed == False:
            return
        # (1) получение разметки таблицы
        # создания объекта рисунка типа Image для разметки таблицы
        self.document_image = DocumentImage(src=self.image_processed_path )
        # распознавание таблицы с ячейками
        table_coords = self.document_image.extract_tables(ocr=None, 
                                                          implicit_rows=False,
                                                          borderless_tables=False,
                                                          min_confidence=50)[0].content        
        # проверка распознавания структуры таблицы
        if len(table_coords) <= 0:
            print(f'Warning:5. Line:{sys._getframe().f_lineno}. Таблица на изображениии не была найдена!')            
            return
        else:
            self.table_extracted = True
        # расположение таблицы, число ячеек и расчетный размер ячейки
        table_params = self.__get_table_pos(table_coords)

        # (2) исправление разметки таблицы
        table_coords_copy = copy.deepcopy(table_coords)
        # проход по строкам словаря разметки    
        col_size_error = False
        for i in range(len(table_coords_copy)):
            # удаление дублирующихся строк
            if i > 0 and table_coords_copy[i] == table_coords_copy[i-1]:
                continue
            # удаление дублирующихся ячеек в столбцах
            shrinked_list = []
            last_cell = None
            last_cell_reserve = None
            next_cell = None
            for j in range(len(table_coords_copy[i])):
                last_cell = next_cell
                next_cell = table_coords_copy[i][j]
                # первая запись в списке заносится отдельно
                if j == 0:
                    last_cell_reserve = next_cell
                    shrinked_list.append(next_cell)                
                    continue            
                # коррекция ширины текущей ячейки за счет объединения со следующей
                if ((j < len(table_coords_copy[0]) - 1)
                    and ((next_cell.bbox.x2 - next_cell.bbox.x1) < 0.7*self.avr_cell_width)
                   ):
                    table_coords_copy[i][j+1].bbox.x1 = next_cell.bbox.x1
                    # восстановление предыдущей записи из копии
                    last_cell = last_cell_reserve
                    continue
                # сравнение координат соседних ячеек
                if (j > 0 and ((next_cell.bbox.x1 != last_cell.bbox.x1)
                              or (next_cell.bbox.x2 != last_cell.bbox.x2))
                ):
                    shrinked_list.append(next_cell)
                    last_cell_reserve = next_cell
            table_coords_copy[i] = shrinked_list
            
            # проверка совпадения размеров столбцов
            if i > 0 and col_size_error == False and len(table_coords_copy[i]) != len(table_coords_copy[i-1]):
                print(len(table_coords_copy[i]), len(table_coords_copy[i-1]), col_size_error)
                col_size_error = True
                print(f'Warning:6. Line:{sys._getframe().f_lineno}. Несовпадение размеров столбцов при разметке таблицы!')
                return
        # уточнение параметров разметки
        self.__get_table_pos(table_coords_copy)
        # сохранение OrderedDict с координатами в строках
        self.table_coords = table_coords_copy
    
    
  
    # функция получения координат ячеек для конкретного столбца\строки
    def get_row_coords(self, num_row, col_coords=True):
        # проверка наличия файла, предобработки изображения, получения разметки таблицы
        if self.image_file_is_exist == False or self.image_file_preprocessed == False or self.table_extracted == False:
            return
        row_coords = []
        # получение координат ячеек для столбца
        if col_coords == True:
            for i in range(len(self.table_coords)):
                row_coords.append(self.table_coords[i][num_row].bbox)
        # получение координат ячеек для строки
        else:
            for cell in self.table_coords[num_row]:
                row_coords.append(cell.bbox)
        return row_coords
    
    

    # функция коррекции записи для типа донорства
    def __improve_type(self, stroke):
        try:
            # замена цифр подходящими символами
            stroke_improved = re.sub('3|8|9', 'в', stroke)
            stroke_improved = re.sub('5|6', 'б', stroke_improved)
            # перевод символов в нижний регистр
            stroke_improved = stroke_improved.lower()
            # очистка лишних знаков
            finded = re.match(r'.*((?:кр|пл|ц)(?:/д)|(?:т/ф))(.*)((?:\(бв\))|(?:\(пл\)))', stroke_improved)
            # проверка частей записи по отдельности
            blood_part = finded.group(1)
            reward_type = finded.group(3)
            if blood_part not in ['кр/д', 'пл/д', 'ц/д', 'т/ф']:
                # коррекция значений с помощью расстояния  Левенштайна
                if len(blood_part) == 3:
                    if lev.distance(blood_part, 'кр/д') < 2:
                        blood_part = 'кр/д'
                    if lev.distance(blood_part, 'пл/д') < 2:
                        blood_part = 'пл/д'
                if len(blood_part) == 2:
                    if lev.distance(blood_part, 'т/ф') < 2:
                        blood_part = 'т/ф'
                    if lev.distance(bood_part, 'ц/д') < 2:
                        blood_part = 'ц/д'
            return finded.group(1) + ' ' + finded.group(3)
        except:
            return "UNKNOWN"
    
    

    # функция оценки адекватности распознавания
    def __check_adequacy(self, stroke, col_type=0):
        # оценка количества
        if col_type == 'quantity':
            try:
                # проверка нахождения количества сданной крови в пределах от 50 до 700 мл
                if 50 <= int(stroke) <= 700:
                    # дополнительная коррекция нормы 450 мл
                    try:
                        re.match(r'(4(?:6|9)(?:0|9|8))', stroke).group(0)
                        stroke = 450
                    except:
                        pass
                    return True, stroke
                else:
                    return False, "UNKNOWN"
            except:
                return False, "UNKNOWN"
        # оценка даты
        elif col_type == 'date':
            # дата приказа о форме справки 
            init_date = datetime.datetime.strptime(r'31.03.2005', '%d.%m.%Y')
            # текущее время
            current_date = datetime.datetime.now()
            date_stroke = ''
            # перебор возможных вариантов записи
            for pattern in [r'%d.%m.%Y', r'%d.%m.%y', r'%d-%m-%Y', r'%d-%m-%y']:
                try:
                    date_stroke = datetime.datetime.strptime(stroke, pattern)
                    break
                except:
                    pass
            # проверка нахождения дат в пределах от даты приказа до текущей
            if date_stroke != '' and (init_date <= date_stroke <= current_date):
                # вывод унифицированного варианта записи напр. 31.12.2020
                return True, date_stroke.strftime('%d.%m.%Y')
            else:
                return False, "UNKNOWN"
        # коррекция записи для типа донорства 
        elif col_type == 'type':
            stroke_corrected = self.__improve_type(stroke)
            if stroke_corrected != "UNKNOWN":
                return True, stroke_corrected
            else:
                return False, "UNKNOWN"
        # вывод без проверки    
        else:
            return True, stroke
    
    

    # функция распознавания содержимого ячеек 
    def recognize_cell(self,
                       cell_coords, 
                       col_type=0,
                       ocr_config=r'--psm 6 --oem 3', 
                       re_mask="re.match(r'(.*)( \..*)', recognized).group(1)", 
                       prc_shrink=0):
        '''
        Данные об области распознавания подаются в виде списка cell_coords в структурах координат Bbox(x1, y1, x2, y2). 
        Для разных типов данных стоит использовать разные конфигурации. 
        Например, для целых чисел r"--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789",
        для даты - r"--psm 13 --oem 1 -c tessedit_char_whitelist=.-0123456789". 
        Подробно о конфигурации можно узнать вызовом !tesseract --help-extra.
        Полный перечень команд конфигурирования вызовом !tesseract --print-parameters.
        Для очистки данных можно подать исполняемую строку регэкспов вида: r"re.match(('.*'), recognized).group(0)".
        Такая строка для целых чисел r"re.match(r'(0*)(\d{2,3})', recognized).group(2)",
        для дат - r"re.match(r'(.*)((?:\d{2})(?:\.|-)(?:\d{2})(?:\.|-)(?:\d{2,4}))(.*)', recognized).group(2)".
        Перед выдачей значения проверяются на адекватность. Некорректные значения записываются как "UNKNOWN".
        '''
        # проверка наличия файла, предобработки изображения, получения разметки таблицы
        if self.image_file_is_exist == False or self.image_file_preprocessed == False or self.table_extracted == False:
            return
        img = cv2.imread(self.image_processed_path)
        text_recognized = []
        for cell in cell_coords:        
            x1 = cell.x1
            y1 = cell.y1
            x2 = cell.x2
            y2 = cell.y2
            # сужение размера ячейки
            x_shift = abs(int((x2-x1)*prc_shrink/100.))
            y_shift = abs(int((y2-y1)*0.5*prc_shrink/100.))
            x1 = x1 + x_shift
            x2 = x2 - x_shift
            y1 = y1 + y_shift
            y2 = y2 - y_shift
            # распознавание содержимого ячейки с преобразованием формата координат (x,y) в формат индексов [y,x]
            recognized = pytesseract.image_to_string(img[y1:y2, x1:x2], lang='rus', config=ocr_config).strip()
            # очистка строки
            finded = recognized
            try:
                finded = eval(re_mask)
            except:
                pass
            # проверка адекватности
            _, finded = self.__check_adequacy(finded, col_type)
            text_recognized.append(finded)
        return text_recognized
    
    

    # функция проверки типа столбца
    def __check_cell_type(self, stroke, type_expected):
        # проверка преобразования строки в целое число
        if type_expected == 'quantity':
            try:
                int(stroke)
                return True
            except:
                return False
        # проверка преобразования строки в дату
        elif type_expected == 'date':
            try:
                datetime.datetime.strptime(stroke, '%d.%m.%Y')
                return True
            except:
                return False
        # проверка наличия в строке кириллических букв
        elif type_expected == 'type':
            try:
                re.match(r'^[а-я]{1}', stroke).group()
                return True
            except:
                return False  
        return None
    
    # функция агрессивной замены отсутствующих значений
    def __get_rough_correcrion(self):
        for i in range(len(self.df_recognised)):
            record = self.df_recognised.iloc[i].copy()
            try:
                # заменая типа данации наиболее вероятными значениями
                if record['type'] == 'UNKNOWN':
                    if int(record['quantity']) >= 350:
                        record['type'] = 'кр/д (бв)'
                    elif  int(record['quantity']) <= 200:
                        record['type'] = 'пл/д (бв)'
                self.df_recognised.iloc[i] = record
            except:
                pass
    
    # функция построение датафрейма из распознанных данных
    def get_dataframe(self):
        '''
        Построение датафрейма из распознанных данных возможно при корректной разметке таблицы в документе. 
        Если она не корректна - датафрейм не формируется. 
        Каждая запись о заборе крови состоит из трех упорядоченных полей, иногда к ним добавляется поле с подписью.
        В выходном датафрейме записи добавляются вниз таблицы из трех полей. 
        '''
        # проверка наличия файла, предобработки изображения, получения разметки таблицы
        if (self.image_file_is_exist == False or self.image_file_preprocessed == False 
            or self.table_extracted == False or self.table_coords is None):
            return
        # проверка корректности разметки
        if (self.flag_split_cell and self.flag_size_incorrected) == True:
            print(f'Warning:7. Line:{sys._getframe().f_lineno}. Таблица размечена не корректно!')
            return None
        else:
            # определение числа полей
            self.num_fields = 0
            if self.n_cols%4 == 0:
                self.num_fields = 4
            elif self.n_cols%3 == 0:
                self.num_fields = 3    
            else:
                print(f'Warning:8. Line:{sys._getframe().f_lineno}. Неизвестный формат таблицы!')
                return        
            # параметры для разных типов данных
            data_type = ['date', 'type', 'quantity']
            data_config = [r'--psm 6 --oem 3 -c tessedit_char_whitelist=.-0123456789', 
                           r'--psm 7 --oem 3',
                           r'--psm 7 --oem 3 -c tessedit_char_whitelist=0123456789']
            data_re = [r"re.match(r'(.*)((?:\d{2})(?:\.|-)(?:\d{2})(?:\.|-)(?:\d{2,4}))(.*)', recognized).group(2)", 
                       r"re.match(('.*'), recognized).group(0)", 
                       r"re.match(r'(0*)(\d{2,3})', recognized).group(2)"]
            df_recognised = pd.DataFrame({'date':[],'type':[],'quantity':[]})
            data_dict = {}
            for i in range(self.n_cols):
                # текущий тип поля
                current_field = (i + self.num_fields) % self.num_fields
                # пропуск поля с подписью
                if self.num_fields == 4 or current_field == 3:
                    continue
                # получение координат ячеек
                row_coords = self.get_row_coords(i, col_coords=True)
                # распознавание по столбцам
                text_recognized = self.recognize_cell(row_coords,
                                                 col_type=data_type[current_field],
                                                 ocr_config=data_config[current_field],
                                                 re_mask=data_re[current_field],
                                                 prc_shrink=-5)
                # подтверждение типа данных от некоторых ячеек
                if sum([self.__check_cell_type(stroke, data_type[current_field]) for stroke in text_recognized]) > 0:
                    # запись текущего столбца по текущему ключу во временный словарь
                    data_type_confirmed = data_type[current_field]
                    data_dict[data_type_confirmed] = text_recognized
                    # завершение формирования блока записей
                    if current_field == 2:                    
                        df_added = pd.DataFrame(data_dict)[2:]
                        # удаление пустой строки в конце блока
                        if current_field > 0:
                            if sum(list(map(lambda x: x == 'UNKNOWN', df_added.iloc[-1]))) == self.num_fields:
                                df_added = df_added.iloc[:-1]
                        # добавление блока в общий датафрйем
                        df_recognised = pd.concat([df_recognised, df_added], axis=0)
                        # очистка словаря
                        data_dict = {}
                        df_recognised.reset_index(drop=True, inplace=True)
            # сохранение результатов распознавания в csv-файл
            if len(df_recognised) > 0:
                # удаление дулирующихся записей
                df_recognised = df_recognised.query("not(date == type == quantity == 'UNKNOWN')").drop_duplicates()
                self.df_recognised = df_recognised                
                # агрессивная подстановка значений
                self.__get_rough_correcrion()
                df_recognised.to_csv(self.csv_recognised_path)
                
    
    
    def estimate_accuracy(self):
        '''
        Точность может быть оценена только при совпадении ID пользователя из csv-файла и названия самого файла,
        а также при совпадении числа распознанных записей и записей из csv-файла.
        '''
        if self.df_recognised is None:
            print(f'Warning:9. Line:{sys._getframe().f_lineno}. Файл csv с распознанными данными не создан!')
            return None
        # проверка наличия csv-файла
        try:
            (pd.read_csv(self.csv_path))
        except:
            print(f'Warning:10. Line:{sys._getframe().f_lineno}. Файл с данными {self.csv_path} отсутствует!')
            return None
        self.df_original = pd.read_csv(self.csv_path)
        # проверка контрольного файла 
        try:
            # проверка вхождения идентификаторов в название файла
            if str(self.df_original['ID пользователя'][0]) in self.csv_path:
                self.accuracy = (self.df_recognised['date'] == self.df_original['Дата донации']).mean()
                print(f'Значение Accuracy по распознавании даты = %.2f' % self.accuracy)
                return self.accuracy
        except:
            print(f'Warning:11. Line:{sys._getframe().f_lineno}. ID пользователя не совпадает с названием файла данных!')
        return None

__Генерация результатов__  
Попробуем применить класс на качественном изображении. Ниже последовательность вызовов для генерации файла результатов распознавания.

In [3]:
# (1) создание экземпляра класса и запуск предобработки
doc = Form405('405/', '231820 .jpg')
# (2) получение разметки и параметров для таблицы
doc.get_table_coords()
# (3) распознавание и сохранение файла с результатами
doc.get_dataframe()

# расчет точности распознавание по полю даты
doc.estimate_accuracy()
doc.df_recognised

,date,type,quantity
0,14.02.2006,кр/д (бв),420
1,11.06.2014,кр/д (бв),350
2,UNKNOWN,кр/д (бв),450
3,13.08.2015,UNKNOWN,UNKNOWN
4,30.10.2015,кр/д (бв),450
5,15.07.2016,кр/д (бв),450
6,11.10.2016,кр/д (бв),450
7,UNKNOWN,кр/д (бв),UNKNOWN
8,21.08.2017,кр/д (бв),450
9,04.08.2018,NaN,450
